In [9]:
import numpy as np
import random
from scipy.optimize import linprog

# Given k authors and m papers, create a random author-paper graph
def assign_papers_to_authors(k, m):
    authors = list(range(k))
    papers = list(range(m))
    author_assignments = {author: [] for author in authors}

    # Step 1: Initial paper assignment ensuring every paper is assigned
    for paper in papers:
        author = random.choice(authors)
        author_assignments[author].append(paper)

    # Step 2: Ensure all authors have at least one paper
    for author in authors:
        if not author_assignments[author]:  # If an author doesn't have a paper
            # Reassign a paper from an author with more than one paper, if possible
            potential_donors = [a for a, p in author_assignments.items() if len(p) > 1]
            if potential_donors:
                donor = random.choice(potential_donors)
                # Ensure donor has at least one paper remaining
                paper = random.choice(author_assignments[donor])
                author_assignments[author].append(paper)
                if m > k:  # Only reassign the paper if there are more papers than authors
                    author_assignments[donor].remove(paper)

    # Step 3: Balance assignments if possible
    for author, assigned_papers in author_assignments.items():
        if len(assigned_papers) < 2:
            potential_papers = [p for p in papers if p not in assigned_papers]
            for paper in potential_papers:
                if sum(paper in p for p in author_assignments.values()) < 2:  # Paper not over-assigned
                    author_assignments[author].append(paper)
                    break  # Exit after assigning one more paper to this author

    return author_assignments


In [15]:
import math

# integrality gap check given number of authors, papers, reviewers, lambda, miu
def integrality_gap_check(number_of_authors, number_of_papers, number_of_reviewers, lambda_value, miu):
  number_of_iterations = 1000
  succeed = 0
  count = 0
  find = False
  for _ in range(number_of_iterations):
    authorship = assign_papers_to_authors(number_of_authors, number_of_papers)
    S = np.random.rand(number_of_reviewers, number_of_papers)
    S = S.flatten()
    A_eq = np.zeros((number_of_papers, number_of_reviewers*number_of_papers))
    b_eq = np.full(number_of_papers, lambda_value)
    for j in range(number_of_papers):
      A_eq[j, j * number_of_reviewers:(j+1)* number_of_reviewers] = 1
    A_ub = np.zeros((number_of_reviewers + number_of_reviewers * number_of_authors, number_of_reviewers*number_of_papers))
    b_ub = np.zeros(number_of_reviewers + number_of_reviewers * number_of_authors)
    for i in range(number_of_reviewers):
      b_ub[i] = miu

    for i in range(number_of_reviewers * number_of_authors):
      b_ub[number_of_reviewers+i] = 1

    for j in range(number_of_reviewers):
      A_ub[j, j::number_of_reviewers] = 1

    for i in range(number_of_reviewers):
      for j in range(number_of_authors):
        for k in authorship[j]:
          A_ub[number_of_reviewers + i * number_of_authors + j][k * number_of_reviewers + i] = 1
    res = linprog(-S, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=[0,1], method='highs')
    res2 = linprog(-S, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=[0,1], method='highs',integrality=1)
    if res.success and res2.success:
      succeed += 1
      o1 = abs(res.fun)
      o2 = abs(res2.fun)
      if abs(o1-o2) > 1e-4:
        find = True
        count += 1
    if find:
      return succeed, count, True

  return succeed, count, False


In [16]:

# integrality gap check for fixed lambda_value and miu
def integrality_gap_check2(lambda_value, miu):
  flag = True
  for i in range(3 * lambda_value, 6 * lambda_value):
    for j in range(3 * miu, 6 * miu):
      number_of_papers = j
      number_of_reviewers = i
      if number_of_reviewers * miu >= number_of_papers * lambda_value:
        flag2 = False
        for _ in range(20):
          number_of_authors = random.randint(number_of_papers, 4 * number_of_papers)
          if integrality_gap_check(number_of_authors, number_of_papers, number_of_reviewers, lambda_value, miu)[2]:
            flag2 = True
            break
        if not flag2:
          flag = False
        if not flag:
          return False

  return True




In [17]:
for i in range(3, 11):
  for j in range(3, 11):
    if integrality_gap_check2(i, j):
      print("lambda =  {} and miu = {} succeed".format(i,j))
    else:
      print("lambda =  {} and miu = {} fail".format(i,j))


lambda =  3 and miu = 3 succeed
lambda =  3 and miu = 4 succeed
lambda =  3 and miu = 5 succeed
lambda =  3 and miu = 6 succeed
lambda =  3 and miu = 7 succeed
lambda =  3 and miu = 8 succeed
lambda =  3 and miu = 9 succeed
lambda =  3 and miu = 10 succeed
lambda =  4 and miu = 3 succeed
lambda =  4 and miu = 4 succeed
lambda =  4 and miu = 5 succeed
lambda =  4 and miu = 6 succeed
lambda =  4 and miu = 7 succeed
lambda =  4 and miu = 8 succeed
lambda =  4 and miu = 9 succeed
lambda =  4 and miu = 10 succeed
lambda =  5 and miu = 3 succeed
lambda =  5 and miu = 4 succeed
lambda =  5 and miu = 5 succeed
lambda =  5 and miu = 6 succeed
lambda =  5 and miu = 7 succeed
lambda =  5 and miu = 8 succeed
lambda =  5 and miu = 9 succeed
lambda =  5 and miu = 10 succeed
lambda =  6 and miu = 3 succeed
lambda =  6 and miu = 4 succeed
lambda =  6 and miu = 5 succeed
lambda =  6 and miu = 6 succeed
lambda =  6 and miu = 7 succeed
lambda =  6 and miu = 8 succeed
lambda =  6 and miu = 9 succeed
lambd